# SPCBViTNet – Epoch 60 Ablation

📌 Part of the ablation study for the submission:  
**"SPCBViTNet: A Self-Parallel Convolutional Block with Vision Transformers for Skin Cancer Classification"**

🔒 Not yet peer-reviewed. For review only.  
📁 [GitHub Repo](https://github.com/diyagoyal31/SPCBViT)


In [1]:
import os
print(os.listdir("/kaggle/input/"))

['skin2skin']


In [2]:
dataset_path = "/kaggle/input/skin2skin/dataset_split_2/"
print(os.listdir(dataset_path))

['MEL', 'SEK', 'SCC', 'ACK', 'NEV', 'BCC']


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [5]:
from torchvision import transforms
from PIL import Image
class SkinCancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform  # Default transform applied in __getitem__
        self.image_paths = []
        self.labels = []

        self.class_names = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.class_names)}

        for cls_name in self.class_names:
            cls_folder = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_folder):
                img_path = os.path.join(cls_folder, img_name)
                if img_path.lower().endswith(('.jpg', '.jpeg', '.png')):  # Supports multiple formats
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load and convert image
        image = Image.open(img_path).convert("RGB")

        # Apply transform (if provided), otherwise apply default ToTensor
        if self.transform:
            image = self.transform(image)
        else:
            image = transforms.ToTensor()(image)

        return image, torch.tensor(label, dtype=torch.long)

In [6]:
import torchvision.transforms.functional as TF
import random

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SKFeaturePyramid(nn.Module):
    def __init__(self, in_channels):
        super(SKFeaturePyramid, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.conv3 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels, in_channels, kernel_size=5, padding=2)
        self.fc = nn.Linear(in_channels, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        f1 = self.conv1(x)
        f2 = self.conv3(x)
        f3 = self.conv5(x)
        f_sum = f1 + f2 + f3
        f_weights = self.softmax(self.fc(f_sum.mean(dim=(2, 3)))).unsqueeze(-1).unsqueeze(-1)
        return f1 * f_weights[:, 0].view(-1, 1, 1, 1) + \
               f2 * f_weights[:, 1].view(-1, 1, 1, 1) + \
               f3 * f_weights[:, 2].view(-1, 1, 1, 1)


class SelfInteractiveAttention(nn.Module):
    def __init__(self, in_channels):
        super(SelfInteractiveAttention, self).__init__()
        self.channel_attn = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attn = nn.Conv2d(in_channels, 1, kernel_size=7, padding=3)

    def forward(self, x):
        ca = self.channel_attn(x) * x
        sa = torch.sigmoid(self.spatial_attn(x)) * x
        return ca + sa


class HigherOrderInteraction(nn.Module):
    def __init__(self, in_channels1, in_channels2, out_channels):
        super(HigherOrderInteraction, self).__init__()
        self.conv1 = nn.Conv2d(in_channels1, out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(in_channels2, out_channels, kernel_size=1)
        self.bilinear_pool = nn.Conv2d(out_channels, out_channels, kernel_size=1)
        self.trilinear_pool = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x1, x2):
        x1 = self.conv1(x1)
        x2 = self.conv2(x2)
        if x1.shape[2:] != x2.shape[2:]:
            x2 = F.interpolate(x2, size=x1.shape[2:], mode='bilinear', align_corners=False)
        return self.bilinear_pool(x1 * x2) + self.trilinear_pool(x1 + x2)


class SPCBViTNet(nn.Module):
    def __init__(self, vit_model, num_classes, patch_grid=(14, 14)):
        super(SPCBViTNet, self).__init__()
        self.vit_model = vit_model
        self.hidden_size = vit_model.config.hidden_size
        self.patch_grid = patch_grid  # E.g. 14x14 for ViT-B/16

        # SPCB modules
        self.sk = SKFeaturePyramid(self.hidden_size)
        self.attn = SelfInteractiveAttention(self.hidden_size)
        self.higher_order = HigherOrderInteraction(self.hidden_size, self.hidden_size, self.hidden_size)

        self.global_avg = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size, 512),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        vit_out = self.vit_model(pixel_values=x).last_hidden_state  # [B, N+1, C]
        cls_token = vit_out[:, 0]  # CLS token
        patch_tokens = vit_out[:, 1:]  # [B, N, C]

        # Convert patch tokens to 2D feature maps
        B, N, C = patch_tokens.size()
        H, W = self.patch_grid
        patch_map = patch_tokens.permute(0, 2, 1).contiguous().view(B, C, H, W)

        # SPCB modules
        f1 = self.sk(patch_map)
        f2 = self.attn(f1)
        f3 = self.higher_order(f1, f2)

        out = self.global_avg(f3).view(B, -1)
        return self.classifier(out)


In [8]:
from transformers import ViTModel

vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

model = SPCBViTNet(vit_model=vit_model, num_classes=6, patch_grid=(14, 14))


2025-05-16 20:38:18.403471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747427898.606109      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747427898.666359      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [9]:
from torchvision import transforms
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# ✅ Data Augmentation for Training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

# ✅ Only Tensor + Normalization for Testing
test_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load full dataset (with neutral transform for indexing)
base_dataset = SkinCancerDataset(root_dir="/kaggle/input/skin2skin/dataset_split_2/", transform=transforms.ToTensor())

NUM_CLASSES = len(base_dataset.class_to_idx)
BATCH_SIZE = 32
EPOCHS = 60
K_FOLDS = 10
LEARNING_RATE = 0.0001

# Extract labels to stratify
targets = [base_dataset[i][1] for i in range(len(base_dataset))]

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
splits = list(skf.split(np.zeros(len(targets)), targets))

all_true_labels = []
all_pred_labels = []
all_probs = []

for fold, (train_idx, test_idx) in enumerate(splits):
    print(f"\n🚀 Training Fold {fold + 1}/{K_FOLDS}")

    # Separate datasets to avoid transform leakage
    train_dataset = SkinCancerDataset(root_dir="/kaggle/input/skin2skin/dataset_split_2/", transform=train_transform)
    test_dataset = SkinCancerDataset(root_dir="/kaggle/input/skin2skin/dataset_split_2/", transform=test_transform)

    train_subset = Subset(train_dataset, train_idx)
    test_subset = Subset(test_dataset, test_idx)

    # Data loaders
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # Initialize model
    model = SPCBViTNet(vit_model=vit_model, num_classes=NUM_CLASSES).to(device)

    # Optimizer & Loss
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = CrossEntropyLoss()

    # Checkpoint path
    checkpoint_path = f"best_model_fold_{fold + 1}.pth"

    # Load checkpoint if available
    best_acc = 0.0
    start_epoch = 0
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_acc = checkpoint["best_acc"]
        print(f"🔄 Resuming from epoch {start_epoch}, best accuracy: {best_acc:.2f}%")

        # Early Stopping Parameters
    patience = 7
    counter = 0

    # Training Loop with Early Stopping
    for epoch in range(start_epoch, EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        loop = tqdm(train_loader, leave=True)
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            loop.set_description(f"Epoch [{epoch+1}/{EPOCHS}]")
            loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

        # Testing Loop
        model.eval()
        test_correct, test_total = 0, 0
        fold_preds, fold_labels, fold_probs = [], [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

                fold_preds.extend(predicted.cpu().numpy())
                fold_labels.extend(labels.cpu().numpy())
                fold_probs.extend(probs.cpu().numpy())

                test_correct += (predicted == labels).sum().item()
                test_total += labels.size(0)

        test_acc = 100 * test_correct / test_total
        print(f"✅ Fold {fold+1}: Test Accuracy = {test_acc:.2f}%")

        # Save checkpoint if accuracy improves
        if test_acc > best_acc:
            best_acc = test_acc
            counter = 0  # reset counter on improvement
            torch.save({
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_acc": best_acc,
            }, checkpoint_path)
            print(f"🎯 New best model saved with accuracy: {best_acc:.2f}%")
        else:
            counter += 1
            print(f"⏳ No improvement. Early stopping counter: {counter}/{patience}")
            if counter >= patience:
                print("🛑 Early stopping triggered.")
                break

    all_true_labels.extend(fold_labels)
    all_pred_labels.extend(fold_preds)
    all_probs.extend(fold_probs)



🚀 Training Fold 1/10


Epoch [1/60]: 100%|██████████| 65/65 [00:43<00:00,  1.51it/s, acc=55.3, loss=0.717]


✅ Fold 1: Test Accuracy = 62.17%
🎯 New best model saved with accuracy: 62.17%


Epoch [2/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=66.2, loss=0.712]


✅ Fold 1: Test Accuracy = 65.65%
🎯 New best model saved with accuracy: 65.65%


Epoch [3/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=70.9, loss=0.643]


✅ Fold 1: Test Accuracy = 75.22%
🎯 New best model saved with accuracy: 75.22%


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=73.9, loss=0.423]


✅ Fold 1: Test Accuracy = 70.00%
⏳ No improvement. Early stopping counter: 1/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=74.6, loss=0.787]


✅ Fold 1: Test Accuracy = 71.30%
⏳ No improvement. Early stopping counter: 2/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=80.4, loss=0.704]


✅ Fold 1: Test Accuracy = 69.57%
⏳ No improvement. Early stopping counter: 3/7


Epoch [7/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=82.5, loss=0.553]


✅ Fold 1: Test Accuracy = 72.17%
⏳ No improvement. Early stopping counter: 4/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=85.2, loss=0.433]


✅ Fold 1: Test Accuracy = 70.00%
⏳ No improvement. Early stopping counter: 5/7


Epoch [9/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=85.8, loss=0.221] 


✅ Fold 1: Test Accuracy = 69.57%
⏳ No improvement. Early stopping counter: 6/7


Epoch [10/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=88.2, loss=0.27]  


✅ Fold 1: Test Accuracy = 75.22%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 2/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=80.9, loss=0.356]


✅ Fold 2: Test Accuracy = 87.39%
🎯 New best model saved with accuracy: 87.39%


Epoch [2/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=88.7, loss=0.319] 


✅ Fold 2: Test Accuracy = 85.65%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=91.2, loss=0.372] 


✅ Fold 2: Test Accuracy = 86.09%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=90.6, loss=0.473] 


✅ Fold 2: Test Accuracy = 86.09%
⏳ No improvement. Early stopping counter: 3/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=91.9, loss=0.145] 


✅ Fold 2: Test Accuracy = 86.09%
⏳ No improvement. Early stopping counter: 4/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=93.2, loss=0.185] 


✅ Fold 2: Test Accuracy = 86.52%
⏳ No improvement. Early stopping counter: 5/7


Epoch [7/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=93.5, loss=0.326] 


✅ Fold 2: Test Accuracy = 86.52%
⏳ No improvement. Early stopping counter: 6/7


Epoch [8/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=93.3, loss=0.107] 


✅ Fold 2: Test Accuracy = 83.48%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 3/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=86.2, loss=0.125] 


✅ Fold 3: Test Accuracy = 97.39%
🎯 New best model saved with accuracy: 97.39%


Epoch [2/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=94.2, loss=0.43]  


✅ Fold 3: Test Accuracy = 94.35%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=94.3, loss=0.048] 


✅ Fold 3: Test Accuracy = 91.74%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=94.4, loss=0.132] 


✅ Fold 3: Test Accuracy = 93.04%
⏳ No improvement. Early stopping counter: 3/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96, loss=0.195]   


✅ Fold 3: Test Accuracy = 93.04%
⏳ No improvement. Early stopping counter: 4/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=94, loss=0.664]    


✅ Fold 3: Test Accuracy = 88.26%
⏳ No improvement. Early stopping counter: 5/7


Epoch [7/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=95.9, loss=0.0325] 


✅ Fold 3: Test Accuracy = 87.83%
⏳ No improvement. Early stopping counter: 6/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96.8, loss=0.293] 


✅ Fold 3: Test Accuracy = 93.04%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 4/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=92.3, loss=0.0604]


✅ Fold 4: Test Accuracy = 95.65%
🎯 New best model saved with accuracy: 95.65%


Epoch [2/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=95.3, loss=0.319] 


✅ Fold 4: Test Accuracy = 96.96%
🎯 New best model saved with accuracy: 96.96%


Epoch [3/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96.5, loss=0.00458]


✅ Fold 4: Test Accuracy = 98.26%
🎯 New best model saved with accuracy: 98.26%


Epoch [4/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97, loss=0.0174]   


✅ Fold 4: Test Accuracy = 95.22%
⏳ No improvement. Early stopping counter: 1/7


Epoch [5/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=94.8, loss=0.0137] 


✅ Fold 4: Test Accuracy = 96.09%
⏳ No improvement. Early stopping counter: 2/7


Epoch [6/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=95.3, loss=0.399]  


✅ Fold 4: Test Accuracy = 92.61%
⏳ No improvement. Early stopping counter: 3/7


Epoch [7/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=96, loss=0.00387]  


✅ Fold 4: Test Accuracy = 93.91%
⏳ No improvement. Early stopping counter: 4/7


Epoch [8/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=96.5, loss=0.0777] 


✅ Fold 4: Test Accuracy = 94.78%
⏳ No improvement. Early stopping counter: 5/7


Epoch [9/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.1, loss=0.0563]


✅ Fold 4: Test Accuracy = 95.22%
⏳ No improvement. Early stopping counter: 6/7


Epoch [10/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.1, loss=0.0166]  


✅ Fold 4: Test Accuracy = 95.22%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 5/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=92.5, loss=0.0628]


✅ Fold 5: Test Accuracy = 95.22%
🎯 New best model saved with accuracy: 95.22%


Epoch [2/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=96, loss=0.0241]   


✅ Fold 5: Test Accuracy = 94.35%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=95.8, loss=0.156]  


✅ Fold 5: Test Accuracy = 91.30%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97, loss=0.0221]   


✅ Fold 5: Test Accuracy = 96.52%
🎯 New best model saved with accuracy: 96.52%


Epoch [5/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.4, loss=0.373]  


✅ Fold 5: Test Accuracy = 94.78%
⏳ No improvement. Early stopping counter: 1/7


Epoch [6/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.8, loss=0.0158] 


✅ Fold 5: Test Accuracy = 96.09%
⏳ No improvement. Early stopping counter: 2/7


Epoch [7/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.1, loss=0.0638]  


✅ Fold 5: Test Accuracy = 95.22%
⏳ No improvement. Early stopping counter: 3/7


Epoch [8/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.6, loss=0.00904]


✅ Fold 5: Test Accuracy = 96.52%
⏳ No improvement. Early stopping counter: 4/7


Epoch [9/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.8, loss=0.0202]  


✅ Fold 5: Test Accuracy = 97.39%
🎯 New best model saved with accuracy: 97.39%


Epoch [10/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.7, loss=0.129]  


✅ Fold 5: Test Accuracy = 92.61%
⏳ No improvement. Early stopping counter: 1/7


Epoch [11/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=98, loss=0.259]     


✅ Fold 5: Test Accuracy = 93.04%
⏳ No improvement. Early stopping counter: 2/7


Epoch [12/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.9, loss=0.00271] 


✅ Fold 5: Test Accuracy = 96.09%
⏳ No improvement. Early stopping counter: 3/7


Epoch [13/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.1, loss=0.0277]  


✅ Fold 5: Test Accuracy = 94.35%
⏳ No improvement. Early stopping counter: 4/7


Epoch [14/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.1, loss=0.738]   


✅ Fold 5: Test Accuracy = 94.78%
⏳ No improvement. Early stopping counter: 5/7


Epoch [15/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.6, loss=0.00306]


✅ Fold 5: Test Accuracy = 95.65%
⏳ No improvement. Early stopping counter: 6/7


Epoch [16/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.8, loss=0.0515]  


✅ Fold 5: Test Accuracy = 93.04%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 6/10


Epoch [1/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=93.9, loss=0.0173] 


✅ Fold 6: Test Accuracy = 97.83%
🎯 New best model saved with accuracy: 97.83%


Epoch [2/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=98.1, loss=0.0688] 


✅ Fold 6: Test Accuracy = 96.96%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=96.2, loss=0.146]  


✅ Fold 6: Test Accuracy = 98.26%
🎯 New best model saved with accuracy: 98.26%


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.1, loss=0.0174] 


✅ Fold 6: Test Accuracy = 97.83%
⏳ No improvement. Early stopping counter: 1/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.4, loss=0.02]   


✅ Fold 6: Test Accuracy = 98.26%
⏳ No improvement. Early stopping counter: 2/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.4, loss=0.00541] 


✅ Fold 6: Test Accuracy = 97.83%
⏳ No improvement. Early stopping counter: 3/7


Epoch [7/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96.7, loss=0.0993] 


✅ Fold 6: Test Accuracy = 97.39%
⏳ No improvement. Early stopping counter: 4/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.7, loss=0.0794] 


✅ Fold 6: Test Accuracy = 90.87%
⏳ No improvement. Early stopping counter: 5/7


Epoch [9/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97, loss=0.0344]    


✅ Fold 6: Test Accuracy = 96.09%
⏳ No improvement. Early stopping counter: 6/7


Epoch [10/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.6, loss=0.0175] 


✅ Fold 6: Test Accuracy = 95.65%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 7/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=93.5, loss=0.096] 


✅ Fold 7: Test Accuracy = 99.13%
🎯 New best model saved with accuracy: 99.13%


Epoch [2/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.2, loss=0.00198] 


✅ Fold 7: Test Accuracy = 99.13%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.8, loss=0.345]  


✅ Fold 7: Test Accuracy = 98.26%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.7, loss=0.377]  


✅ Fold 7: Test Accuracy = 98.70%
⏳ No improvement. Early stopping counter: 3/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97, loss=0.0132]   


✅ Fold 7: Test Accuracy = 94.78%
⏳ No improvement. Early stopping counter: 4/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96.6, loss=0.00591]


✅ Fold 7: Test Accuracy = 98.26%
⏳ No improvement. Early stopping counter: 5/7


Epoch [7/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.7, loss=0.0733] 


✅ Fold 7: Test Accuracy = 97.83%
⏳ No improvement. Early stopping counter: 6/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.9, loss=0.0155] 


✅ Fold 7: Test Accuracy = 98.26%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 8/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=94.1, loss=0.00552]


✅ Fold 8: Test Accuracy = 99.13%
🎯 New best model saved with accuracy: 99.13%


Epoch [2/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.8, loss=0.171]  


✅ Fold 8: Test Accuracy = 96.52%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=96.3, loss=0.366]  


✅ Fold 8: Test Accuracy = 96.52%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=98.1, loss=0.406]   


✅ Fold 8: Test Accuracy = 97.83%
⏳ No improvement. Early stopping counter: 3/7


Epoch [5/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.3, loss=0.1]    


✅ Fold 8: Test Accuracy = 98.70%
⏳ No improvement. Early stopping counter: 4/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.5, loss=0.000687]


✅ Fold 8: Test Accuracy = 98.26%
⏳ No improvement. Early stopping counter: 5/7


Epoch [7/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=99.1, loss=0.00756] 


✅ Fold 8: Test Accuracy = 98.70%
⏳ No improvement. Early stopping counter: 6/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.3, loss=0.0867]  


✅ Fold 8: Test Accuracy = 98.70%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 9/10


Epoch [1/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=95, loss=0.722]     


✅ Fold 9: Test Accuracy = 98.25%
🎯 New best model saved with accuracy: 98.25%


Epoch [2/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.9, loss=0.0956]  


✅ Fold 9: Test Accuracy = 99.13%
🎯 New best model saved with accuracy: 99.13%


Epoch [3/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.7, loss=0.116]  


✅ Fold 9: Test Accuracy = 99.56%
🎯 New best model saved with accuracy: 99.56%


Epoch [4/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=96.2, loss=0.304]  


✅ Fold 9: Test Accuracy = 99.13%
⏳ No improvement. Early stopping counter: 1/7


Epoch [5/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.2, loss=0.0256] 


✅ Fold 9: Test Accuracy = 97.82%
⏳ No improvement. Early stopping counter: 2/7


Epoch [6/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=97.8, loss=0.00472] 


✅ Fold 9: Test Accuracy = 99.56%
⏳ No improvement. Early stopping counter: 3/7


Epoch [7/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.2, loss=0.00586] 


✅ Fold 9: Test Accuracy = 100.00%
🎯 New best model saved with accuracy: 100.00%


Epoch [8/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=98.9, loss=0.031]   


✅ Fold 9: Test Accuracy = 98.69%
⏳ No improvement. Early stopping counter: 1/7


Epoch [9/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.9, loss=0.167]   


✅ Fold 9: Test Accuracy = 98.25%
⏳ No improvement. Early stopping counter: 2/7


Epoch [10/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.2, loss=0.00579] 


✅ Fold 9: Test Accuracy = 99.56%
⏳ No improvement. Early stopping counter: 3/7


Epoch [11/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.5, loss=0.191]   


✅ Fold 9: Test Accuracy = 97.82%
⏳ No improvement. Early stopping counter: 4/7


Epoch [12/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.9, loss=0.294]   


✅ Fold 9: Test Accuracy = 93.45%
⏳ No improvement. Early stopping counter: 5/7


Epoch [13/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.8, loss=0.0446]  


✅ Fold 9: Test Accuracy = 96.07%
⏳ No improvement. Early stopping counter: 6/7


Epoch [14/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.8, loss=0.198]   


✅ Fold 9: Test Accuracy = 98.25%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.

🚀 Training Fold 10/10


Epoch [1/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=94.5, loss=0.0883] 


✅ Fold 10: Test Accuracy = 99.56%
🎯 New best model saved with accuracy: 99.56%


Epoch [2/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=98.2, loss=0.199]   


✅ Fold 10: Test Accuracy = 98.25%
⏳ No improvement. Early stopping counter: 1/7


Epoch [3/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.8, loss=0.0215] 


✅ Fold 10: Test Accuracy = 98.69%
⏳ No improvement. Early stopping counter: 2/7


Epoch [4/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=98.6, loss=0.000319]


✅ Fold 10: Test Accuracy = 99.13%
⏳ No improvement. Early stopping counter: 3/7


Epoch [5/60]: 100%|██████████| 65/65 [00:42<00:00,  1.55it/s, acc=99, loss=0.000367]  


✅ Fold 10: Test Accuracy = 97.38%
⏳ No improvement. Early stopping counter: 4/7


Epoch [6/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=98.2, loss=0.00875] 


✅ Fold 10: Test Accuracy = 97.38%
⏳ No improvement. Early stopping counter: 5/7


Epoch [7/60]: 100%|██████████| 65/65 [00:42<00:00,  1.54it/s, acc=97.5, loss=0.171]   


✅ Fold 10: Test Accuracy = 96.94%
⏳ No improvement. Early stopping counter: 6/7


Epoch [8/60]: 100%|██████████| 65/65 [00:41<00:00,  1.55it/s, acc=97.9, loss=0.00949] 


✅ Fold 10: Test Accuracy = 98.25%
⏳ No improvement. Early stopping counter: 7/7
🛑 Early stopping triggered.



📊 Final Evaluation Across All Folds:


precision    recall  f1-score   support

         ACK     0.9370    0.9370    0.9370       730
         BCC     0.9185    0.9467    0.9324       845
         MEL     0.8772    0.9615    0.9174        52
         NEV     0.9741    0.9262    0.9496       244
         SCC     0.9048    0.7917    0.8444       192
         SEK     0.9292    0.9489    0.9389       235

    accuracy                         0.9291      2298
   macro avg     0.9235    0.9187    0.9200      2298
weighted avg     0.9293    0.9291    0.9287      2298



In [10]:

from sklearn.metrics import confusion_matrix
import numpy as np  # Ensure numpy is also imported
print("\n📊 Final Evaluation Across All Folds:")

cm = confusion_matrix(all_true_labels, all_pred_labels)
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(8, 6))
sns.heatmap(cm_percentage, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=base_dataset.class_to_idx.keys(), 
            yticklabels=base_dataset.class_to_idx.keys())

plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Final Confusion Matrix Across All Folds (Percentage-wise)")
plt.show()

class_report = classification_report(all_true_labels, all_pred_labels, target_names=base_dataset.class_to_idx.keys(), digits=4)
print(class_report)

plt.figure(figsize=(10, 8))
for i in range(NUM_CLASSES):
    fpr, tpr, _ = roc_curve(np.array(all_true_labels) == i, np.array(all_probs)[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {list(base_dataset.class_to_idx.keys())[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Final ROC Curve Across All Folds")
plt.legend(loc="lower right")
plt.show()
